In [1]:
%load_ext autoreload
%autoreload 2

from ml_collections import config_dict
import jax.random as random

from src.utils.training import setup_model, get_dataset_splits, train_loop
import src.utils.input as i

In [2]:
rng = random.PRNGKey(0)
data_rng, model_rng = random.split(rng)

In [3]:
def get_config(low_angle, high_angle) -> config_dict.ConfigDict:
    config = config_dict.ConfigDict()

    config.dataset = 'MNIST'
    # config.data_dir = '~/data'
    config.shuffle_buffer_size = 50_000
    config.repeat_after_batch = True  # NOTE: ordering of PP and repeat is important!
    config.train_split = 'train[10000:]'
    config.pp_train = f'value_range(-1, 1)|random_rotate({low_angle}, {high_angle}, fill_value=-1)|keep(["image"])'  # decode(1)|
    config.val_split = 'train[:10000]'
    config.pp_eval = f'value_range(-1, 1)|random_rotate({low_angle}, {high_angle}, fill_value=-1)|keep(["image"])'  # |random_rotate({low_angle}, {high_angle}, fill_value=-1)


    # config.optim_name = 'adamw'
    # config.optim = config_dict.ConfigDict()
    # config.optim.weight_decay = 1e-4
    # config.learning_rate = 1e-4

    # config.lr_schedule_name = 'warmup_cosine_decay_schedule'
    # config.lr_schedule = config_dict.ConfigDict()
    # config.lr_schedule.peak_value = 3 * config.learning_rate
    # config.lr_schedule.end_value = 1/3 * config.learning_rate
    # num_train, _ = train_val_split_sizes(METADATA['num_train'][config.dataset_name], config.val_percent)
    # num_batches_per_epoch = ceil(num_train / config.batch_size)
    # config.lr_schedule.decay_steps = config.epochs * num_batches_per_epoch
    # config.lr_schedule.warmup_steps = int(0.2 * config.lr_schedule.decay_steps)

    # config.model_name = 'VAE'
    # config.model = config_dict.ConfigDict()
    # config.model.latent_dim = 32

    # config.β = 10
    # config.β_schedule_name = 'cosine_decay_schedule'
    # config.β_schedule = config_dict.ConfigDict()
    # config.β_end_value = 1
    # config.β_schedule.alpha = config.β_end_value / config.β
    # config.β_schedule.decay_steps = config.epochs * num_batches_per_epoch

    # config.model.encoder = config_dict.ConfigDict()
    # config.model.encoder.posterior = 'hetero-diag-normal'
    # config.model.encoder.hidden_dims = [64, 128]  # [64, 128, 256]

    # config.model.decoder = config_dict.ConfigDict()
    # config.model.decoder.likelihood = 'iso-normal'
    # config.model.decoder.σ_min = 1e-2
    # config.model.decoder.hidden_dims = list(reversed(config.model.encoder.hidden_dims))
    # config.model.decoder.hidden_dims = [384, 192, 96, 48]
    # config.model.decoder.image_shape = METADATA['image_shape'][config.dataset_name]

    return config

In [4]:
config = get_config(45, 0)

In [5]:
train_ds, val_ds, _ = get_dataset_splits(config, data_rng, 256, 256)

In [6]:
ds = i.start_input_pipeline(train_ds, 1)
next(ds)['image'].shape

(1, 256, 28, 28, 1)